In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [52]:
!pip install xgboost


In [56]:
# Bibliotecas
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score
from xgboost import XGBClassifier

# caminho drive
file_path = '/content/drive/MyDrive/dados cartão/creditcard.csv'

df = pd.read_csv(file_path)

# Verificar os dados
print(df.info())

# Normalização
scaler = StandardScaler()
X = scaler.fit_transform(df.drop('Class', axis=1))
y = df['Class']

# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Definir o modelo XGBoost
model = XGBClassifier(eval_metric='logloss')

# Definir os parâmetros para Grid Search com valores reduzidos
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [3, 6],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.8, 1.0]
}

# Grid Search com um número menor de combinações e iterações
grid = GridSearchCV(model, param_grid, scoring='roc_auc', cv=2, verbose=1)
grid_result = grid.fit(X_train, y_train)

# Resultados do Grid Search
print("Melhor Grid Search: %f usando %s" % (grid_result.best_score_, grid_result.best_params_))

# Treinar e avaliar o modelo otimizado do Grid Search
best_model = grid_result.best_estimator_
y_pred_best = best_model.predict(X_test)
print("AUC-ROC Otimizado com Grid Search:", roc_auc_score(y_test, y_pred_best))

# Definir parâmetros para Random Search com valores reduzidos
param_dist = {
    'n_estimators': [50, 100],
    'max_depth': [3, 6],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.8, 1.0]
}

# Random Search com menos iterações
random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=5, scoring='roc_auc', cv=2, verbose=1, random_state=42)
random_result = random_search.fit(X_train, y_train)

# Resultados do Random Search
print("Melhor Random Search: %f usando %s" % (random_result.best_score_, random_result.best_params_))

# Treinar e avaliar o modelo otimizado do Random Search
best_model_random = random_result.best_estimator_
y_pred_best_random = best_model_random.predict(X_test)
print("AUC-ROC Otimizado com Random Search:", roc_auc_score(y_test, y_pred_best_random))

# Modelo Original
original_model = XGBClassifier(eval_metric='logloss')
original_model.fit(X_train, y_train)
y_pred_original = original_model.predict(X_test)

# Comparação dos Resultados
print("\nComparação dos Resultados:")
print(f"AUC-ROC Modelo Original: {roc_auc_score(y_test, y_pred_original)}")
print(f"AUC-ROC Melhor Modelo Otimizado com Grid Search: {roc_auc_score(y_test, y_pred_best)}")
print(f"AUC-ROC Melhor Modelo Otimizado com Random Search: {roc_auc_score(y_test, y_pred_best_random)}")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

# Análise e Ajuste do Modelo de Detecção de Fraudes em Cartões de Crédito

1. **Treinamento do Modelo Original:**
 Foi feito um modelo XGBoost treinado com os dados fornecidos. O desempenho inicial do modelo foi avaliado utilizando a métrica AUC-ROC.

**Resultados:** O AUC-ROC do modelo original foi de 0.908.

**Próximos Passos:** Validar a eficácia do modelo original e considerar ajustes adicionais para melhorar o desempenho.





2. **Definição e Implementação de Hiperparâmetros:**

Foram definidos intervalos de valores para os hiperparâmetros do XGBoost, Grid Search e Random Search para encontrar a melhor configuração.

**Resultados do Grid Search:** O melhor desempenho obtido foi um AUC-ROC de 0.912 com os parâmetros {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'subsample': 0.8}.


**Resultados do Random Search:** O melhor desempenho foi idêntico ao do Grid Search, com um AUC-ROC de 0.912 com os mesmos parâmetros.


**Próximos Passos:** Ampliar o intervalo de valores para os hiperparâmetros ou explorar técnicas adicionais de otimização, como Bayesian Optimization.


3. **Comparação de Resultados:**

AUC-ROC do Modelo Original: 0.908


AUC-ROC do Melhor Modelo Otimizado com Grid Search: 0.912


AUC-ROC do Melhor Modelo Otimizado com Random Search: 0.912


**Próximos Passos:** Realizar uma análise detalhada das métricas de desempenho, como precisão, recall e F1-score. Avaliar a significância das melhorias observadas.


4. **Análise Adicional dos Dados:**

Embora a normalização dos dados tenha sido realizada, uma análise mais aprofundada pode revelar padrões importantes ou problemas potenciais.

**Próximos Passos:** Conduzir uma análise exploratória dos dados para identificar correlações, padrões, e verificar a presença de valores ausentes ou desequilíbrios na classe.



5. **Validação Cruzada Avançada:**

A validação cruzada realizada foi básica. Implementar uma validação cruzada mais robusta pode ajudar a garantir que o modelo seja generalizável.


**Próximos Passos:** Aplicar K-Fold Cross-Validation para garantir que o modelo performe bem e evitar sobreajuste.

